In [1]:
import cosima_cookbook as cc
import xarray as xr
import cartopy.crs as ccrs

from dask.distributed import Client
import numpy as np

In [2]:
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:33207 Dashboard: /proxy/33645/status,Cluster Workers: 8 Cores: 8 Memory: 100.00 GiB


In [3]:
session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')

In [15]:
expt='01deg_jra55v140_iaf'
start = '2001-09-01 00:00:01'
end = '2001-12-31 23:59:00'

Load data and slice to the time frame required in a single step

In [6]:
[f.ncfile for f,v in cc.querying._ncfiles_for_variable(expt='01deg_jra55v140_iaf', variable='u', 
                       session=session, frequency='1 daily',
                       start_time=start, 
                       end_time=end)]

['output172/ocean/ocean-3d-u-1-daily-mean-ym_2001_01.nc',
 'output172/ocean/ocean-3d-u-1-daily-mean-ym_2001_02.nc',
 'output172/ocean/ocean-3d-u-1-daily-mean-ym_2001_03.nc',
 'output173/ocean/ocean-3d-u-1-daily-mean-ym_2001_04.nc',
 'output173/ocean/ocean-3d-u-1-daily-mean-ym_2001_05.nc',
 'output173/ocean/ocean-3d-u-1-daily-mean-ym_2001_06.nc',
 'output174/ocean/ocean-3d-u-1-daily-mean-ym_2001_07.nc',
 'output174/ocean/ocean-3d-u-1-daily-mean-ym_2001_08.nc',
 'output174/ocean/ocean-3d-u-1-daily-mean-ym_2001_09.nc',
 'output175/ocean/ocean-3d-u-1-daily-mean-ym_2001_10.nc',
 'output175/ocean/ocean-3d-u-1-daily-mean-ym_2001_11.nc',
 'output175/ocean/ocean-3d-u-1-daily-mean-ym_2001_12.nc']

In [7]:
u = cc.querying.getvar(expt='01deg_jra55v140_iaf', variable='u', 
                       session=session, frequency='1 daily',
                       start_time=start, 
                       end_time=end).sel(time=slice(start,end)).chunk({'st_ocean':75, 'yu_ocean':540, 'xu_ocean':720})

In [8]:
u

,Array,Chunk
Bytes,0.97 TiB,111.24 MiB
Shape,"(365, 75, 2700, 3600)","(1, 75, 540, 720)"
Count,1177137 Tasks,9125 Chunks
Type,float32,numpy.ndarray


Create a mask of all the bottom cells by masking a single time slice of the u velocity data with itself but shifted "down" one cell in st_ocean.

In a second step turn it into a boolean array and rename.

In [9]:
bot_mask = u.isel(time=0).where(~xr.ufuncs.isfinite(u.isel(time=0).shift({'st_ocean':-1})))
bot_mask = ~xr.ufuncs.isnan(bot_mask).to_dataset(name='umask')
bot_mask

<xarray.Dataset>
Dimensions:   (st_ocean: 75, xu_ocean: 3600, yu_ocean: 2700)
Coordinates:
  * xu_ocean  (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 ... 89.92 89.96 90.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
    time      datetime64[ns] 2001-01-01T12:00:00
Data variables:
    umask     (st_ocean, yu_ocean, xu_ocean) bool dask.array<chunksize=(75, 540, 720), meta=np.ndarray>

In [14]:
for month, u_month in u.resample({'time': '1M'}):
    print(month, u_month)
    u_month.where(bot_mask.umask).sum('st_ocean').to_dataset(name='ubot').to_netcdf('u_bot_{d}.nc'.format(d=month))

2001-01-31T00:00:00.000000000 <xarray.DataArray 'u' (time: 31, st_ocean: 75, yu_ocean: 2700, xu_ocean: 3600)>
dask.array<getitem, shape=(31, 75, 2700, 3600), dtype=float32, chunksize=(1, 75, 540, 720), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 ... 89.92 89.96 90.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 2001-01-01T12:00:00 ... 2001-01-31T12:00:00
Attributes:
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  sea_water_x_velocity
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 365, nv: 2)>\ndask...
2001-02-28T00:00:00.000000000 <xarray.DataArray 'u' (time: 28, st_ocean: 75, yu_ocean: 2700, xu_ocea

In [16]:
v = cc.querying.getvar(expt='01deg_jra55v140_iaf', variable='v', 
                       session=session, frequency='1 daily',
                       start_time=start, 
                       end_time=end).sel(time=slice(start,end)).chunk({'st_ocean':75, 'yu_ocean':540, 'xu_ocean':720})

In [17]:
for month, v_month in v.resample({'time': '1M'}):
    print(month, v_month)
    v_month.where(bot_mask.umask).sum('st_ocean').to_dataset(name='vbot').to_netcdf('v_bot_{d}.nc'.format(d=month))

2001-09-30T00:00:00.000000000 <xarray.DataArray 'v' (time: 30, st_ocean: 75, yu_ocean: 2700, xu_ocean: 3600)>
dask.array<getitem, shape=(30, 75, 2700, 3600), dtype=float32, chunksize=(1, 75, 540, 720), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 ... 89.92 89.96 90.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 2001-09-01T12:00:00 ... 2001-09-30T12:00:00
Attributes:
    long_name:      j-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  sea_water_y_velocity
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 122, nv: 2)>\ndask...
2001-10-31T00:00:00.000000000 <xarray.DataArray 'v' (time: 31, st_ocean: 75, yu_ocean: 2700, xu_ocea